In [1]:
import torch

In [2]:
corpus = ['king is a strong man', 
          'queen is a wise woman', 
          'boy is a young man',
          'girl is a young woman',
          'prince is a young king',
          'princess is a young queen',
          'man is strong', 
          'woman is pretty',
          'prince is a boy will be king',
          'princess is a girl will be queen']

In [3]:
def get_set(corpus):
    words = []
    for line in corpus:
        for word in line.split(' '):
            words.append(word)
    return set(words)

In [4]:
words = get_set(corpus)
words

{'a',
 'be',
 'boy',
 'girl',
 'is',
 'king',
 'man',
 'pretty',
 'prince',
 'princess',
 'queen',
 'strong',
 'will',
 'wise',
 'woman',
 'young'}

## Skip-gram part

In [5]:
word2int = {}
for i, word in enumerate(words):
    word2int[word] = i
word2int

{'a': 0,
 'strong': 1,
 'wise': 2,
 'princess': 3,
 'prince': 4,
 'king': 5,
 'queen': 6,
 'is': 7,
 'woman': 8,
 'boy': 9,
 'girl': 10,
 'will': 11,
 'be': 12,
 'pretty': 13,
 'young': 14,
 'man': 15}

In [6]:
sentences = []
for sentence in corpus:
    sentences.append(sentence.split(' '))
    
sentences

[['king', 'is', 'a', 'strong', 'man'],
 ['queen', 'is', 'a', 'wise', 'woman'],
 ['boy', 'is', 'a', 'young', 'man'],
 ['girl', 'is', 'a', 'young', 'woman'],
 ['prince', 'is', 'a', 'young', 'king'],
 ['princess', 'is', 'a', 'young', 'queen'],
 ['man', 'is', 'strong'],
 ['woman', 'is', 'pretty'],
 ['prince', 'is', 'a', 'boy', 'will', 'be', 'king'],
 ['princess', 'is', 'a', 'girl', 'will', 'be', 'queen']]

In [11]:
data = []
window_size = 2
for sentence in sentences:
    for i, target in enumerate(sentence):
        for word in sentence[max(0, i-1):min(len(sentence)-1, i + 1)]:
            if target != word:
                data.append([target, word])
data

[['is', 'king'],
 ['a', 'is'],
 ['strong', 'a'],
 ['man', 'strong'],
 ['is', 'queen'],
 ['a', 'is'],
 ['wise', 'a'],
 ['woman', 'wise'],
 ['is', 'boy'],
 ['a', 'is'],
 ['young', 'a'],
 ['man', 'young'],
 ['is', 'girl'],
 ['a', 'is'],
 ['young', 'a'],
 ['woman', 'young'],
 ['is', 'prince'],
 ['a', 'is'],
 ['young', 'a'],
 ['king', 'young'],
 ['is', 'princess'],
 ['a', 'is'],
 ['young', 'a'],
 ['queen', 'young'],
 ['is', 'man'],
 ['strong', 'is'],
 ['is', 'woman'],
 ['pretty', 'is'],
 ['is', 'prince'],
 ['a', 'is'],
 ['boy', 'a'],
 ['will', 'boy'],
 ['be', 'will'],
 ['king', 'be'],
 ['is', 'princess'],
 ['a', 'is'],
 ['girl', 'a'],
 ['will', 'girl'],
 ['be', 'will'],
 ['queen', 'be']]